In [ ]:
import requests
import random
import os
import time
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

To access the COD API We need to generate a random device id and then register it. I suppose this is a security measure? We'll need this device Id in future API calls where authentication is required.

We'll also get an access token in return that we will need.

In [ ]:
device_id = hex(random.getrandbits(128)).lstrip("0x")
payload =  {"deviceId": device_id}
resp = requests.post('https://profile.callofduty.com/cod/mapp/registerDevice', json=payload)

access_token = resp.json()['data']['authHeader']

In [ ]:
headers = {
    "Authorization": f"Bearer {access_token}",
    "x_cod_device_id" : device_id,
}

data = {'email': os.environ.get('COD_EMAIL'), 'password': os.environ.get('COD_PW')}

resp_login = requests.post('https://profile.callofduty.com/cod/mapp/login', headers=headers, json=data)

In [ ]:
resp_login.cookies

In [ ]:
resp_login.json()

In [ ]:
match_id = "3726192899756849207"
team_mates = {"Vio": "14688938695022220470", "Bublous": "1860799185863297602"}
#, "Uncarriable": '5291500827578550226'
#team_mates = {"Huskerrs": "11946786255256953231", "Symfuhny": "4864369508081987227", "ZLaner": "5499179043424889056"}
#team_mates = {"BookaShade": "16503230311151154188"}
#team_mates = {"Huskerrs": "11946786255256953231", "ZLaner": "5499179043424889056", "OakBoi": "1506652682349281517", "AydaN": "6466570850733519449"}
#team_mates = {"ZLaner": "5499179043424889056", "Destroy": "13121215955392487462", "Trash_Fue": "9516497865377061135", "SuperEvan": "3099898901879856705"}

In [ ]:
lobby_data = requests.get('https://my.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/battle/fullMatch/wz/' + match_id + '/it')

In [ ]:
lobby_players = lobby_data.json()['data']['allPlayers']
lobby_players

In [ ]:
kds = []
lobby_players_full = []
team = ""
teamPlacement = ""
team_stats = {}
for player in lobby_players:
    time.sleep(0.2)
    if player['player']['uno'] in team_mates.values():
        team = player['player']['team']
        team_placement = player['playerStats']['teamPlacement']
        team_stats[player['player']['username']] = [player['playerStats']['kills']]
    resp = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/' + str(player['player']['uno']) + '/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
    kds.append(resp.json()['data']['lifetime']['mode']['br']['properties']['kdRatio'])
    lobby_players_full.append(resp.json()['data'])
    print(kds)
kds

In [ ]:
plt.hist(kds, density=True, bins=50)
plt.xlabel("KD")
plt.ylabel("#Players")

In [ ]:
avg = round(sum(kds)/len(kds), 2)
math.floor(len(kds)/2)
median = round(sorted(kds)[math.floor(len(kds)/2)], 2)
kd_min = round(min(kds), 2)
kd_max = round(max(kds), 2)
top_ten = [round(kd, 2) for kd in sorted(kds)[math.floor(len(kds) * (80/100)):len(kds)]]
print("Game ID: " + match_id)
print("Team Placement: " + str(team_placement))
print("Team Kills")
print(team_stats)
print("Lobby Stats")
print("Average KD: " + str(avg))
print("Median KD: " + str(median))
print("Worst KD: " + str(kd_min))
print("Best KD:" + str(kd_max))
print("Top 20% KD: " + str([x for x in top_ten]))


In [ ]:
sorted_kds = sorted(kds)

In [ ]:
sorted_kds = sorted_kds[:-4]
adj_avg = round(sum(sorted_kds)/len(sorted_kds), 2)
print(sorted_kds[-1])
print(len(sorted_kds))
print(adj_avg)

In [ ]:
max(kds)

In [ ]:
min(kds)

In [ ]:
np.var(kds)

In [ ]:
resp_profile = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/12035878875057403705/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
resp_profile.json()

In [ ]:
resp_profile.json()['data']['lifetime']['mode']['br']['properties']['kdRatio']

In [ ]:
'https://my.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/uno/uno/4066176156082092197/matches/wz/start/0/end/0/details' \
--header 'Cookie: ACT_SSO_COOKIE=Set by test scripts; ACT_SSO_COOKIE_EXPIRY=1591153892430; atkn=Set by test scripts;'

In [ ]:
resp_profile = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/psn/gamer/Viioozz/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
resp_profile.json()